#Deep MNIST para Expertos

TensorFlow es una poderosa libreria para hacer calculos numericos a gran escala. Una de las tareas que realiza esta libreria es implemntar y entrenar deep neural networks. En este tutorial se aprende los bloques basicos para construir un modelo TensorFlow mientras se construye un clasificador deep convolutional MNIST.

#Preparacion

Antes de crear el modelo primero se debe cargar el dataset MNIST y despues iniciar una sesion TensorFlow.

#Cargar la informacion de MNIST

Para cargar la informacion se incluye un script que automaticamente descarga e importa el dataset de MNIST. Cuando se ejecuta este script se crea un directorio con en nombre de 'MNIST_data' en el cual se guardan los archivos con los datos.

a continuacion se presentan las lineas que ejecutan dicho script:

In [62]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


La variable mnist contiene los arreglos de tipo numpy de entrenamiento, validacion y testing. Ademas provee una funcion que permite iterar atraves de los minibatches de datos los cuales se utilizan mas adelante.

#Comenzar una sesion interactiva en TensorFlow

TensorFlow se basa en un eficiente backend de C++ para hacer los calculos. La conexion para este backend se llama sesion. El uso mas comun para programas TensorFlow es primero crear un grafo y despues iniciar una sesion.

Aqui se utiliza por conveniciencia la clase InteractiveSession la cual hace que TensorFlow sea aun mas flexible acerca de la estructura del codigo Tambien permite intercalar operaciones las cuales construyen un grafo de calculos con el cual se ejecutara el grafo. Esto es particularmente conveniente cuando se trabaja con contextos interactivos como iPython. 

A continuacion se inicia la sesion:

In [67]:
import tensorflow as tf
sess = tf.InteractiveSession()

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fe72c5462d0>> ignored


#Computacion Grafica (Cambiar este nombre!) grafo computacional ..

Para realizar calculos numericos eficientes en python usualmente se utilizan librerias como NumPy que hacen operaciones costosas como las multiplicacion de matrices fuera de Python utilizando codigo eficiente implementado en otro lenguaje. Desafortunadamente existe una gran cantidad de overhead al cambiar de una operacion a otra.

TensorFlow utiliza librerias implementadas en otros lenguajes pero evitando el overhead. Provee un grafo con operaciones interactivas las cuales corren fuera de python.

El rol de codigo en python es construir un grafo externo y dictarle que partes de el grafo se van a utilizar.

# Construir el Modelo de la Regresion Softmax

En esta seccion se construye el modelo de la regresion softmax con una sola capa. En la siguiente seccion se extiene el caso de la regresion softmax con multicapas en una convolutional network.

#Placeholders

Para comenzar a construir un grafo computacional se crean nodos para las imagenes de entrada y salidas para las clases.

In [3]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

Aqui x e y_ no tienen valores especificos ya que son placeholders esto significa que se les asignara un valor de entrada cuando se le pida a TensorFlow ejecutar alguna operacion.

Las imagenes de entrada x consisten de un tensor 2d de numeros de punto flotante. Aqui se les asigna un tamano de [None, 784], en donde 784 es la dimension de una imagen de MNIST y None indica que la primera dimension corresponde al tamano del batch este puede tener cualquier tamano. La salida de las clases es -_ el cual consiste de un tensor 2d donde cada renglon es un vector "one-hot 10-dimensional" indicando cual es el digito de la clase a la cual corresponde la imagen a tratar.

#Variables

Ahora se definen los pesos W y los bias B para el modelo. Se pueden pensar que son entradas adicionales, pero TensorFlow tiene una manera de manejar este tipo de entradas como variables. Una variable es un valor que vive en el grafo computacional de TensorFlow . Estas pueden ser usadas e incluso modificadas por calculos. Comunmente en aplicaciones de machine learning los modelos utilizan parametros variables.

A continuacion se declaran dichas bariables para el peso y el bias:

In [4]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

Primero se pasa el valor inicial para cada parametro en la llamada a la funcion tf.variable. En este caso se inicializan w y b con tensores llenos de 0's. W es una matriz de 784 x 10 (Porque se tienen 784 entradas y 10 salidas) y b es un vector de dimension 10 (Porque se tienen 10 clases).

Antes de que las variables se utilicen dentro de una sesion se deben inicializar usando esa sesion. Este paso toma los valores iniciales (En este caso los tensores llenos de 0's ) e inicializa todas las variables a la vez

A continuacion se inicializan las variables:

In [5]:
sess.run(tf.initialize_all_variables())

#Prediciendo la clase y la funcion de costo

Ahora se puede implementar el modelo de regresion. Solo toma una linea de codigo! Se multiplica el vector de imagenes de entrada x por la matriz de pesos W y se agrega el bias b con esto se ejecutan las probabilidades de softmax asignandola a cada clase.

In [6]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

La funcion de costo se minimiza durante el entrenamiento. La funcion de costo a utilizar es cross-entropy  la cual se utiliza la salida conocida y la de la prediccion del modelo.

In [7]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

Como nota tf.reduce_sum suma todas las imagenes en el minibatch asi como todas las clases. Se calcula la funcion de costo para todo el minibatch.

#Entrenando el Modelo

Ya que se ha definido el modelo y la funcion de costo para entrenamiento se pasa a entrenar el modelo con TensorFlow. TensorFlow cuenta con el grafo computacional el cual se puede usar para encontrar el gradiente del costo respecto a las variables. TensorFlow cuenta con varios algoritmos de optimizacion. Como ejemplo se utiliza el descenso de gradiente con un paso de longitud de 0.01 para minimizar la funcion de costo cross entropy.

In [68]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

NameError: name 'cross_entropy' is not defined

Lo que hizo TensorFlow en una sola linea es agregar nuevas operaciones al grafo computacional. Estas operaciones  incluyen la operacion de calcular el gradiente, actualizar los pasos y aplicar los pasos actualizados a los parametros.

Cuando se ejecuta trains_step se aplicara descenso de gradiente y actualizara los parametros. Por lo tanto se puede entrenar el modelo ejecutando varias veces train_step.

In [69]:
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

NameError: name 'train_step' is not defined

Cada iteracion del entrenamiento carga 50 ejemplos de entrenamiento. Cuando se ejecuta la operacion train_step se utiliza feed_dict esto para reemplazar los placeholder de los tensores x e y_ con los ejemplos de entrenamiento. Como nota se puede reemplazar cualquier tensor en el grafo computacional utilizando feed_dict no esta restringido a solo placeholders. 

#Evaluando el Modelo

Que tan bien lo hizo el modelo?

Primero se va a revisar si el modelo predijo la etiqueta correcta. tf.argmax es una funcion muy util que regresa el indice mas alto de una posicion en el tensor sobre un eje. Por ejemplo tf.argmax(y, 1) es la etiqueta que se considera mas probable en el modelo mientras que tf.argmax(y_, 1) es la etiqueta correcta. Se utiliza tf.equal para revisar si la prediccion concuerta con la verdadera etiqueta.


In [70]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

NameError: name 'y' is not defined

La instruccion anterior regresa una lista de valores bool. Para determinar que fraccion de datos son correctos se aplica un cast a la lista cambiando los valores bool a valores de punto flotante y luego se calcula la media. Por ejemplo supongamos que tenemos la siguiente lista [True, Flase, True, True] aplicando el cast se convierte en la siguiente lista [1, 0, 1, 1] y despues se saca la media que en este caso es 0.75.

A continuacion se presenta lo antes descrito por la siguiente linea de codigo:

In [71]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

NameError: name 'correct_prediction' is not defined

Finalmente se puede evaluar la exactitud del modelo en los datos que se van a probar. El valor obtenido se debe de encontrar entre el 91% de exactitud.

In [73]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

NameError: name 'accuracy' is not defined

#Construir una Computational Network Multicapas (reepensar titulo)

Obtener una exactitud de 91% en los datos de MNIST es una mala exactitud en esta se seccion se utiliza un modelo mas sofisticado al cual se le llama convolutional neural network. Con este modelo se obtiene una exactitud alrededor del 99.2% de exactitud.

#Inicializacion de los pesos (Le falta)

Para crear este modelo se necesitan muchos pesos y bias. A continuacion se presentan dos funciones utiles que inicializan los pesos y bias.

In [74]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

#Convolucion y Pooling (Le falta)

TensorFlow tambien provee mucha flexibilidad en las operaciones de convolucion y pooling. Como se manejan los limites? cual es el tamanio del paso? En este ejemplo se utiliza la version vanilla que significa que es la version simple. Las Convoluciones utilizan un paso a la vez y zero padded por lo que la salida es del mismo tamanio que la entrada. El pooling maneja maximo bloques de 2 x 2. Para mantener el codigo limpio se crean las siguientes funciones que realizan las operaciones antes mencionadas.
 

In [75]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


#Primer Capa Convolucional

Ahora se puede implementar la primer capa la cual consite de la convolucion seguida del max pooling. La convolucion calcula 32 caracteristicas para cada parte de 5 x 5. El peso del tensor tendra la forma de [5, 5, 1, 32]. Las primeras dos dimensiones es el tamano de la parte, el siguiente numebro representa los canales de entrada y el ultimo numero los canales de salida. Se tendra un vector del bias para un componente para cada salida del canal.

In [76]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

Para aplicar la capa primero se debe de redimensionar x a un tensor de dimension 4 en donde la segunda y tercera dimension corresponden al anchura y altura de la imagen el ultimo numero corresponde al color de los canales.

In [77]:
x_image = tf.reshape(x, [-1,28,28,1])

NameError: name 'x' is not defined

Ahora se convoluciona x_image con el tensor de peso, se agrega el bias, se aplica la funcion RELU y finalmente mas pool.

In [78]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

NameError: name 'x_image' is not defined

#Segunda Capa Convolucional

Para mantener un orden para construir una deep network se apilan varias capas de este tipo. La segunda capa consiste de 64 caracteristicas para cada parte de 5 x 5.

In [80]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

NameError: name 'h_pool1' is not defined

#Capa Densamente Conectada

Ahora el tamanio de las imagenes se han reducido a 7 x 7 Se agrega una capa totalmente conectada con 1024 neuronas las cuales procesan toda la imagen. Se redimensiona el tensor de la capa de pooling a un batch de vectores y se multiplica por la matriz de pesos ademas se agrega el bias y por ultimo se aplica la funcion ReLu.

In [81]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

NameError: name 'h_pool2' is not defined

#Dropout

Para reducir el sobreajuste se aplicara dropout antes del readout de la capa. Se crea un placeholder para la probabilidad de la salida de la neurona la cual se mantiene durante el dropout. Esto permite activar el dropout durante el entrenamiento y desactivarlo durante el testing. TensorFlow's tf.nn.dropout automaticamente maneja las salidas de las neuronas ademas les agrega una mascara entonces dropout funciona bien sin ningun otro ajuste adicional.

In [82]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

NameError: name 'h_fc1' is not defined

#Capa de Readout

Finalmente se agrega la capa de softmax tal como la que se presento anteriormente

In [83]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

NameError: name 'h_fc1_drop' is not defined

#Entrenar y Evaluar el Modelo

Que tan bien lo hace el modelo? Para entrenar y evaluar el modelo se usa un codigo parecido al de la capa softmax. Las diferencias son que se reemplaza el algoritmo de optimizacion de descenso de gradiente por uno mas sofisticado el cual se llama ADAM ademas se incluira el parametro adicional keep_prob en feed_dict para controlar la taza de dropout y despues se imprime un mensaje cada 100 iteraciones para ver la exactitud del proceso de entrenamiento.

In [84]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

NameError: name 'y_' is not defined

Para finalizar la ejecucion del codigo anterior debe regresar una exactidud de aproximadamente 99.2%.